In [ ]:
# 이미지 배경 제거 -> 흰색 노란색 머리의 경우 검은 배경으로 사진 찍을 것을 요청
#     -> 배경 제거 완료했고, 첫 이미지의 배경이 검은색인지 흰색인지만 판단하기
#     -> 이것도 그냥 적도록 하면 안되나?
#     -> 판단도 참 애매한데
#     -> 근데 이게 아예 검은색 아예 흰색이라 실제 배경으로 테스트 해봐야할 듯





# 이미지 배경제거
# 얼굴 좌표를 기준으로 사진 커팅
# 새로운 사진(얼굴 좌표로 커팅 된 배경되지 않은) 불러오기
# 좌표 비교 -> (제거된 자표 & !얼굴범위) ? 제거 : continue
# 배경 제거 및 얼굴 좌표 생성 완료
# 이후에 시간이 남으면 사용자에게 얼굴 추출된 사진을 보여주고 잘못된 좌표 선택 가능하도록 업데이트


# Exception
# 머리가 은색이나 흰색이면 어떡해? -> 동방예의지국인데 은색 머리..?
# 완전 생 노란색 머리도 지워지네 http://image.auction.co.kr/itemimage/fd/1c/12/fd1c12696.jpg -> 노란 머리가 말이야? 
# 무조건 흰색 배경 앞에서 찍을 것 !!!


###### ++ 대머리는 어떡해? -> 출시할 거 아니니까 괜찮아



# 떠올라버렸어 이거 장난아니야
# 밝은 머리의 경우 -> 검은 배경으로
# 어두운 머리의 경우 -> 흰색 배경으로

# --> 배경이 어두울 경우 색반전
# --> 배경제거
# --> 배경제거 버전을 기준으로 기존 사진에서 배경 다시 제거
# --> 얼굴 검출
# --> 검출되지 않은 좌표 && 제거되지 않은 좌표 == 모발
# /* 색반전 코드
# import cv2


# img = cv2.imread("lenna.png")
# out = img.copy()
# out = 255 - out
# cv2.imshow("original", img)
# cv2.imshow("flip", out)
# cv2.waitKey(0)
# */








# 생각생각 생각생각
# 1. 머리색 추측 
# /* 배경 제거 후 얼굴 좌표 위 일정 영역을 탐색하면 지워졌는지 안 지워졌는지 알 수 있음
#    혹은 탐색한 영역의 평균 색 (지워진 영역을 제외하고)으로 탐지
# */
# 검은머리 ~ 갈색머리 등 어두운 색일 경우엔 임계값 낮출 시 모발 동공만 남음 (흰색 배경 기준)
# -> 쉽게 모발 동공을 구할 수 있음
# -> 렌즈 끼고 찍은 사람 윌스미스 마렵네
# 흰색머리 ~ 노란 머리 등 밝은 색일 경우엔 어떡해?
# -> 여행 전 염색이 필요합니다.
# ->
# 오늘 아침에 보라색 머리도 보고 반은 검은색 반은 노란색도 봤는데 경우의 수가 너무 많은데?
# 2. 통계에서 일정 수치가 지워졌더라도 남은 수치로 충분히 추측이 가능함
# 지워진 부분을 두고 얼굴 좌표 기준 위쪽 애들 비교
# -> 아무리 머리가 흰색이어도 진짜 백발이 아닌 이상 어느정도 남기에 남은 머리로 퍼스널컬러를 추측
# -> 이게 최곤가?
# 3. 일정 임계값을 정한 뒤 그로부터 5~10씩 계속 높이기
# -> 얼굴 좌표 기준으로 일정 비율의 범위를 정하고 계속해서 임계값을 임계값을 높임
# -> 처음엔 계속해서 지워진 머리가 채워지기 시작
# -> 첫 낮은 임계값 기준 + 임의로 정한 일정 비율 범위 내 모발을 채워나감


# 하ㄹㄹ-르 라는 애가 있는데 이거 써볼까? 설명글이 너무 길어서 난독증 오는데 할 수 있을까?

In [1]:
import numpy as np
import cv2

In [10]:
import numpy as np
import cv2

# imageUrl = 'img/test_img_brown_1.jpg'
# imageUrl = 'img/test_img_black_1.jpg'
imageUrl = 'img/test_img_black_3.jpg'
# imageUrl = 'img/test_img_black_2.jpg'
imageUrl = 'img/eye_resize.jpg'

# 이미지 불러오기
img = cv2.imread(imageUrl)

print(type(img))

# img = 255 - img # 이미지 배경이 검인지 흰인지 판단하는 함수 필요 

# 변환 graky
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 임계값 조절
mask = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]
# mask = cv2.adaptiveThreshold(gray, 10, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 3, 2)[1]
# (gray, 120, cv2.ADAPTIVE_THRESH_MEAN_C, 255, cv2.THRESH_BINARY, 15, 2)[1]
# (img,  255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2. THRESH_BINARY,15,2)
# (img  ,255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,15,2)

# mask
mask = 255 - mask

# morphology 적용
# borderconstant 사용
kernel = np.ones((3,3), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

# anti-alias the mask
# blur alpha channel
mask = cv2.GaussianBlur(mask, (0,0), sigmaX=2, sigmaY=2, borderType = cv2.BORDER_DEFAULT)

# linear stretch so that 127.5 goes to 0, but 255 stays 255
mask = (2*(mask.astype(np.float32))-255.0).clip(0,255).astype(np.uint8)

# put mask into alpha channel
result = img.copy()
result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
result[:, :, 3] = mask


# result = 255 - result

h, w, c = result.shape

print('h : ', h)
print('w : ', w)

# for x in range(0, w):
#     for y in range(0, h):
#         if(result[y, x][0] != 255 & result[y, x][1] != 255 & result[y, x][2] != 255):
#             result[y, x][0] = 255 - result[y, x][0]
#             result[y, x][1] = 255 - result[y, x][1]
#             result[y, x][2] = 255 - result[y, x][2]

            
# return result
            
# 저장
resultimg_url = 'img/ttt.png'
cv2.imwrite(resultimg_url, result)

<class 'numpy.ndarray'>
h :  79
w :  72


True

In [14]:
import numpy as np
import cv2

def cutBackground_darkHair(img):
    # 변환 graky
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 임계값 조절
    mask = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]

    # mask
    mask = 255 - mask

    # morphology 적용
    # borderconstant 사용
    kernel = np.ones((3,3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # anti-alias the mask
    # blur alpha channel
    mask = cv2.GaussianBlur(mask, (0,0), sigmaX=2, sigmaY=2, borderType = cv2.BORDER_DEFAULT)

    # linear stretch so that 127.5 goes to 0, but 255 stays 255
    mask = (2*(mask.astype(np.float32))-255.0).clip(0,255).astype(np.uint8)

    # put mask into alpha channel
    result = img.copy()
    result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
    result[:, :, 3] = mask

    return result

resultimg_url = 'img/eye_result.jpg'
# resultimg_url = 'img/test_img_black_3.jpg'
# resultimg = 'translated_yellowHair.png'

# 이미지 불러오기
resultimg = cv2.imread(resultimg_url)
img_eye = cutBackground_darkHair(resultimg)

resultimg_url = 'img/' + 'eye_cut' + '_result.png'
cv2.imwrite(resultimg_url, np.float32(img_eye))

True

In [79]:
def getEyes(img, personal):
#     json_facial_points = json.loads(str(res.json()['result']['faces'])[1:-1].replace('\'', '\"'))['facial_points']
#     h_o, w_o, c = img.shape
#     polygon_face = []
    
#     for position in json_facial_points['left_eye']:
#         point = (int(position[0]*w_o), int(position[1]*h_o))
#         polygon_face.append(point)

#     # convert to numpy (for convenience)
#     result_img = np.asarray(img)

#     # create new image ("1-bit pixels, black and white", (width, height), "default color")
#     mask_img = Image.new('1', (result_img.shape[1], result_img.shape[0]), 0)

#     ImageDraw.Draw(mask_img).polygon(polygon_face, outline=1, fill=1)
#     mask = np.array(mask_img)

#     # assemble new image (uint8: 0-255)
#     new_img_array = np.empty(result_img.shape, dtype='uint8')

#     # copy color values (RGB)
#     new_img_array[:,:,:3] = result_img[:,:,:3]

#     # filtering image by mask
#     new_img_array[:,:,0] = new_img_array[:,:,0] * mask
#     new_img_array[:,:,1] = new_img_array[:,:,1] * mask
#     new_img_array[:,:,2] = new_img_array[:,:,2] * mask

#     # back to Image from numpy
#     result_img = Image.fromarray(new_img_array, "RGB")

#     resultimg_url = 'img/' + 'eye' + '_result.jpg'
#     cv2.imwrite(resultimg_url, np.float32(result_img))
    
    h_o, w_o, c = img.shape
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
#     personal_low = [personal[0]-10, personal[1], personal[2]]
#     personal_high = [255, 255, 255]
#     low = lab2rgb(personal_low)
#     high = lab2rgb(personal_high)
    
#     img_cuteye = cv2.inRange(img, (low[0], low[1], low[2]), (high[0], high[2], high[2])) # 이거 기준 하얀색은 제거
#     img_cuteye = cv2.inRange(img, (240, 240, 240), (255, 255, 255)) # 이거 기준 하얀색은 제거
    
#     resultimg_url = 'img/' + 'test_cuteye' + '_result.jpg'
#     cv2.imwrite(resultimg_url, np.float32(img_cuteye))
    
#     print('img : ', type(img))
#     print('img_cut : ', type(img_cutHead))
    

#     for position_x in range(0, w_o):
#         for position_y in range(0, h_o):
# #             print((img_cuteye[position_y,position_x]!= (0, 0, 0))[0] == True)
#             condition = img_cuteye[position_y, position_x] != (0, 0, 0)
#             if (condition[0] and condition[1] and condition[2]:
#                 print("before : ", img_cuteye[position_y,position_x])
#                 img[position_y, position_x][0] = 0
#                 img[position_y, position_x][1] = 0
#                 img[position_y, position_x][2] = 0
    
    print('==========================================================================')
    

#     for position_x in range(0, w_o):
#         for position_y in range(0, h_o):
#             print("before : ", img[position_y,position_x])
    
    h_o, w_o, c = img.shape
    for position_x in range(0, w_o):
        for position_y in range(0, h_o):
            condition_r = img[position_y, position_x][0] < 70
            condition_g = img[position_y, position_x][1] < 70
            condition_b = img[position_y, position_x][2] < 70
            if (condition_r == False and condition_g == False and condition_b == False):
                img[position_y, position_x] = [0, 0, 0]
            
    return np.float32(img)

resultimg_url = 'img/eye_resize.jpg'
# resultimg_url = 'img/eye_result.jpg'
# resultimg_url = 'img/test_img_black_3.jpg'

# 이미지 불러오기
resultimg = cv2.imread(resultimg_url)
img_eye = getEyes(resultimg, (255, 255, 255))

resultimg_url = 'img/' + 'eye_cut' + '_result.png'
cv2.imwrite(resultimg_url, np.float32(img_eye))

resultimg_url = 'img/' + 'eye_cut' + '_result.jpg'
cv2.imwrite(resultimg_url, np.float32(img_eye))

True

In [1]:
resultimg_url = 'img/eye_result.jpg'
# resultimg = 'translated_yellowHair.png'

# 이미지 불러오기
resultimg = cv2.imread(resultimg_url)

resultimg = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
# resultimg[:, :, 3] = resultimg

h, w, c = resultimg.shape

for x in range(0, w):
    for y in range(0, h):
        if(resultimg[x, y][0] != 0 & resultimg[x, y][1] != 0 & resultimg[x, y][2] != 0):
            result[x, y][0] = img[x, y][0]
            result[x, y][1] = img[x, y][1]
            result[x, y][2] = img[x, y][2]


new_image_url = "/img/new_image.png"
cv2.imshow(new_image_url, resultimg)
cv2.imwrite(new_image_url, resultimg)

NameError: name 'cv2' is not defined

In [46]:
img = new_image

img = img[:, :, ::-1]

new_image_url = "/img/new_image.png"
cv2.imwrite(new_image_url, img)

TypeError: Expected Ptr<cv::UMat> for argument 'img'

In [3]:
# 이미지 자르기 (x1 y1 x2 y2)

cropped_image = pil_image.crop((480, 520, 580, 580))
cropped_image

'4.5.1'